--- Day 14: One-Time Pad ---

In order to communicate securely with Santa while you're on this mission, you've been using a one-time pad that you generate using a pre-agreed algorithm. Unfortunately, you've run out of keys in your one-time pad, and so you need to generate some more.

To generate keys, you first get a stream of random data by taking the MD5 of a pre-arranged salt (your puzzle input) and an increasing integer index (starting with 0, and represented in decimal); the resulting MD5 hash should be represented as a string of lowercase hexadecimal digits.

However, not all of these MD5 hashes are keys, and you need 64 new keys for your one-time pad. A hash is a key only if:

    It contains three of the same character in a row, like 777. Only consider the first such triplet in a hash.
    One of the next 1000 hashes in the stream contains that same character five times in a row, like 77777.

Considering future hashes for five-of-a-kind sequences does not cause those hashes to be skipped; instead, regardless of whether the current hash is a key, always resume testing for keys starting with the very next hash.

For example, if the pre-arranged salt is abc:

    The first index which produces a triple is 18, because the MD5 hash of abc18 contains ...cc38887a5.... However, index 18 does not count as a key for your one-time pad, because none of the next thousand hashes (index 19 through index 1018) contain 88888.
    The next index which produces a triple is 39; the hash of abc39 contains eee. It is also the first key: one of the next thousand hashes (the one at index 816) contains eeeee.
    None of the next six triples are keys, but the one after that, at index 92, is: it contains 999 and index 200 contains 99999.
    Eventually, index 22728 meets all of the criteria to generate the 64th key.

So, using our example salt of abc, index 22728 produces the 64th key.

Given the actual salt in your puzzle input, what index produces your 64th one-time pad key?


In [1]:
import hashlib

In [2]:
filepath = "..\\data\\input_day_14.txt"
test1 = "..\\test\\test14_1.txt"
test2 = "..\\test\\test14_2.txt"

In [3]:
def read_input(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    return lines

In [4]:
def generate_hash(base_seed, i):
    seed = base_seed + str(i)
    result = hashlib.md5(seed.encode())
    return result.hexdigest()

In [5]:
def find_first_triplet(string):
    
    lowest, lowest_char = 1000, "_"
    for char in set(string):
            
        trip = char * 3
        if trip in string:
            #print(i , trip, result)
            current = string.find(trip)
            if current < lowest :
                lowest = current
                lowest_char = char
        
    return lowest_char

In [15]:
def find_five(string,  triplets, index):
    
    total = 0
    for char in set(string):

        fiver = char*5

        if fiver in string and char in triplets:
            total += "".join(triplets).count(char)
            for i, t in enumerate(triplets):
                if t=="char":
                    print(index-1000+i)
    return total
                    

In [16]:
def day14a(filepath):
    
    base_seed = read_input(filepath)[0]
    print(base_seed)
    
    triplets = []
    triplets_long = []
    keys = 0
    i = 0
        
    while True:
                
        result = generate_hash(base_seed, i)
                     
        # check for 5 of the same characters continuous
        keys += find_five(result, triplets, i)
        #print(i)
            
        # find the first triplet
        triplets.append(find_first_triplet(result))
        
        if len(triplets)>1000:
            triplets.pop(0)       
        if keys >= 64:
            print(i)
            return i
        i += 1                    
    #print(keys_index)

In [17]:
def test14a():
    
    # Check if the hashes are generated correctly
    assert "cc38887a5" in generate_hash("abc", 18)
    assert "eee" in generate_hash("abc", 39)
    assert "999" in generate_hash("abc", 92)
    print("Passed all hashes checks")
    
    
    # Check if triplets are generated correctly
    assert find_first_triplet(generate_hash("abc", 18)) == "8"
    assert find_first_triplet(generate_hash("abc", 39)) == "e"
    assert find_first_triplet(generate_hash("abc", 92)) == "9"
    print("Passed all triplet checks")
    
    # Check if fivers are found correctly 
    
    
    print("Passed all checks")

In [18]:
test14a()

Passed all hashes checks
Passed all triplet checks
Passed all checks


In [19]:
day14a(test1)

abc
22804


22804

In [20]:
day14a(filepath)

yjdafjpo
25531


25531

In [12]:
generate_hash("abc", 39)

'347dac6ee8eeea4652c7476d0f97bee5'

In [13]:
generate_hash("abc", 18)

'0034e0923cc38887a57bd7b1d4f953df'

In [14]:
import hashlib
import sys
import re

def get_hash_part1(salt, index):
    m = hashlib.md5()
    m.update(salt.encode("utf-8"))
    m.update(str(index).encode("utf-8"))
    return m.hexdigest()

def get_hash_part2(salt, index):
    h = get_hash_part1(salt, index)

    for i in range(2016):
        h = hashlib.md5(h.encode("utf-8")).hexdigest()

    return h

def find_five_repeats(hashes, ch):
    quintuple = ch * 5

    for h in hashes:
        if h.find(quintuple) != -1:
            return True

    return False    

def solve(salt, get_hash):
    triple = re.compile(r'(.)\1\1')

    hashes = [get_hash(salt, x) for x in range(1001)]

    index = 0
    found = 0

    while True:
        md = triple.search(hashes.pop(0))
        if md and find_five_repeats(hashes, md.group(1)):
            found += 1
            print(index)
            if found >= 64:
                break

        index += 1
        hashes.append(get_hash(salt, index + len(hashes)))

    return index

salt = "abc"

print("Part 1:", solve(salt, get_hash_part1))
#print("Part 2:", solve(salt, get_hash_part2))

39
92
110
184
291
314
343
385
459
461
489
771
781
887
955
1144
5742
5781
5783
6016
6093
6219
7833
7858
7918
7937
8042
8045
8183
8189
8205
8232
8375
8407
8431
8503
8517
8626
8672
8730
8811
9497
9536
13268
13439
13479
13560
13663
15758
15883
16187
16342
16479
20087
20371
20582
20635
20669
21908
21927
21978
22023
22193
22728
Part 1: 22728
